# Optimizing Model

## Agenda

- Quantizing model
- Converting the Bert model with torch script

In [92]:
import pathlib
import sklearn
import datasets
import pandas as pd
import torch

import numpy as np
import transformers
import os
import json
from ts.utils.util  import map_class_to_label


In [2]:
model_dir ='../artifacts/model/amazon/'

<function torch.nn.functional.softmax(input: torch.Tensor, dim: Union[int, NoneType] = None, _stacklevel: int = 3, dtype: Union[int, NoneType] = None) -> torch.Tensor>

In [3]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_dir
                                                                        #, num_labels=len(labels)
                                                                       #, label2id=label2id
                                                                       
                                                                       )

In [4]:
model.config

DistilBertConfig {
  "_name_or_path": "../artifacts/model/amazon/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "ABIS_DRUGSTORE",
    "1": "ABIS_LAWN_AND_GARDEN",
    "2": "ACCESSORY",
    "3": "ACCESSORY_OR_PART_OR_SUPPLY",
    "4": "AUTO_ACCESSORY",
    "5": "BABY_PRODUCT",
    "6": "BACKPACK",
    "7": "BATTERY",
    "8": "BEAUTY",
    "9": "BED",
    "10": "BED_FRAME",
    "11": "BENCH",
    "12": "BISS",
    "13": "BOOT",
    "14": "BRACELET",
    "15": "BREAD",
    "16": "CABINET",
    "17": "CELLULAR_PHONE_CASE",
    "18": "CHAIR",
    "19": "CHARGING_ADAPTER",
    "20": "CLEANING_AGENT",
    "21": "CLOCK",
    "22": "CLOTHES_HANGER",
    "23": "COFFEE",
    "24": "COMPUTER_ADD_ON",
    "25": "COMPUTER_COMPONENT",
    "26": "DAIRY_BASED_DRINK",
    "27": "DESK",
    "28": "DRINKING_CUP",
    "29": "EARRING",
    "30": "EDIBLE_

In [5]:
labels = model.config.label2id
labels

{'ABIS_DRUGSTORE': 0,
 'ABIS_LAWN_AND_GARDEN': 1,
 'ACCESSORY': 2,
 'ACCESSORY_OR_PART_OR_SUPPLY': 3,
 'AUTO_ACCESSORY': 4,
 'BABY_PRODUCT': 5,
 'BACKPACK': 6,
 'BATTERY': 7,
 'BEAUTY': 8,
 'BED': 9,
 'BED_FRAME': 10,
 'BENCH': 11,
 'BISS': 12,
 'BOOT': 13,
 'BRACELET': 14,
 'BREAD': 15,
 'CABINET': 16,
 'CELLULAR_PHONE_CASE': 17,
 'CHAIR': 18,
 'CHARGING_ADAPTER': 19,
 'CLEANING_AGENT': 20,
 'CLOCK': 21,
 'CLOTHES_HANGER': 22,
 'COFFEE': 23,
 'COMPUTER_ADD_ON': 24,
 'COMPUTER_COMPONENT': 25,
 'DAIRY_BASED_DRINK': 26,
 'DESK': 27,
 'DRINKING_CUP': 28,
 'EARRING': 29,
 'EDIBLE_OIL_VEGETABLE': 30,
 'ELECTRONIC_ADAPTER': 31,
 'FILE_FOLDER': 32,
 'FINEEARRING': 33,
 'FINENECKLACEBRACELETANKLET': 34,
 'FINEOTHER': 35,
 'FINERING': 36,
 'FLAT_SCREEN_DISPLAY_MOUNT': 37,
 'FLAT_SHEET': 38,
 'FOOD_SERVICE_SUPPLY': 39,
 'FURNITURE': 40,
 'FURNITURE_COVER': 41,
 'GROCERY': 42,
 'HANDBAG': 43,
 'HARDWARE': 44,
 'HARDWARE_HANDLE': 45,
 'HAT': 46,
 'HEADBOARD': 47,
 'HEADPHONES': 48,
 'HEALTH_PERSON

In [6]:
id2label = model.config.id2label

In [98]:
id2label

{0: 'ABIS_DRUGSTORE',
 1: 'ABIS_LAWN_AND_GARDEN',
 2: 'ACCESSORY',
 3: 'ACCESSORY_OR_PART_OR_SUPPLY',
 4: 'AUTO_ACCESSORY',
 5: 'BABY_PRODUCT',
 6: 'BACKPACK',
 7: 'BATTERY',
 8: 'BEAUTY',
 9: 'BED',
 10: 'BED_FRAME',
 11: 'BENCH',
 12: 'BISS',
 13: 'BOOT',
 14: 'BRACELET',
 15: 'BREAD',
 16: 'CABINET',
 17: 'CELLULAR_PHONE_CASE',
 18: 'CHAIR',
 19: 'CHARGING_ADAPTER',
 20: 'CLEANING_AGENT',
 21: 'CLOCK',
 22: 'CLOTHES_HANGER',
 23: 'COFFEE',
 24: 'COMPUTER_ADD_ON',
 25: 'COMPUTER_COMPONENT',
 26: 'DAIRY_BASED_DRINK',
 27: 'DESK',
 28: 'DRINKING_CUP',
 29: 'EARRING',
 30: 'EDIBLE_OIL_VEGETABLE',
 31: 'ELECTRONIC_ADAPTER',
 32: 'FILE_FOLDER',
 33: 'FINEEARRING',
 34: 'FINENECKLACEBRACELETANKLET',
 35: 'FINEOTHER',
 36: 'FINERING',
 37: 'FLAT_SCREEN_DISPLAY_MOUNT',
 38: 'FLAT_SHEET',
 39: 'FOOD_SERVICE_SUPPLY',
 40: 'FURNITURE',
 41: 'FURNITURE_COVER',
 42: 'GROCERY',
 43: 'HANDBAG',
 44: 'HARDWARE',
 45: 'HARDWARE_HANDLE',
 46: 'HAT',
 47: 'HEADBOARD',
 48: 'HEADPHONES',
 49: 'HEALTH_PE

In [100]:
id2label_str = {str(key): value for key, value in id2label.items()}
id2label_str

{'0': 'ABIS_DRUGSTORE',
 '1': 'ABIS_LAWN_AND_GARDEN',
 '2': 'ACCESSORY',
 '3': 'ACCESSORY_OR_PART_OR_SUPPLY',
 '4': 'AUTO_ACCESSORY',
 '5': 'BABY_PRODUCT',
 '6': 'BACKPACK',
 '7': 'BATTERY',
 '8': 'BEAUTY',
 '9': 'BED',
 '10': 'BED_FRAME',
 '11': 'BENCH',
 '12': 'BISS',
 '13': 'BOOT',
 '14': 'BRACELET',
 '15': 'BREAD',
 '16': 'CABINET',
 '17': 'CELLULAR_PHONE_CASE',
 '18': 'CHAIR',
 '19': 'CHARGING_ADAPTER',
 '20': 'CLEANING_AGENT',
 '21': 'CLOCK',
 '22': 'CLOTHES_HANGER',
 '23': 'COFFEE',
 '24': 'COMPUTER_ADD_ON',
 '25': 'COMPUTER_COMPONENT',
 '26': 'DAIRY_BASED_DRINK',
 '27': 'DESK',
 '28': 'DRINKING_CUP',
 '29': 'EARRING',
 '30': 'EDIBLE_OIL_VEGETABLE',
 '31': 'ELECTRONIC_ADAPTER',
 '32': 'FILE_FOLDER',
 '33': 'FINEEARRING',
 '34': 'FINENECKLACEBRACELETANKLET',
 '35': 'FINEOTHER',
 '36': 'FINERING',
 '37': 'FLAT_SCREEN_DISPLAY_MOUNT',
 '38': 'FLAT_SHEET',
 '39': 'FOOD_SERVICE_SUPPLY',
 '40': 'FURNITURE',
 '41': 'FURNITURE_COVER',
 '42': 'GROCERY',
 '43': 'HANDBAG',
 '44': 'HARDWARE'

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
                model_dir
            )

In [8]:
query = 'men shoes for work'
query  = "men vitamins"

query  = "herbal TEA"


In [9]:
res = tokenizer.encode_plus(query, return_tensors="pt")
res

{'input_ids': tensor([[  101, 27849,  5572,   102]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [10]:
tokenizer.convert_ids_to_tokens(res['input_ids'][0])

['[CLS]', 'herbal', 'tea', '[SEP]']

In [11]:
%%time
model_res = model(**res)
model_res

CPU times: user 161 ms, sys: 5.02 ms, total: 166 ms
Wall time: 20.8 ms


SequenceClassifierOutput(loss=None, logits=tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
          -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
          -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
          -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
          -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
          -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
           4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
          -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
         -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
          -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
          -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
          -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
         

torch.Size([1, 112])

In [102]:
topk = 20
ps = torch.nn.functional.softmax(model_res.logits, dim=1)
probs, classes = torch.topk(ps, topk, dim=1)
probs = probs.tolist()
classes = classes.tolist()

In [103]:
probs, classes

([[0.7232240438461304,
   0.25969767570495605,
   0.006111669819802046,
   0.0060967495664954185,
   0.0011588569032028317,
   0.000731807027477771,
   0.0005764953093603253,
   0.0004914401797577739,
   0.0002766131074167788,
   0.00024830058100633323,
   0.00020814224262721837,
   0.00019130650616716594,
   0.00018323035328648984,
   0.00010350668162573129,
   0.00010080193169414997,
   9.626983955968171e-05,
   5.7873698096955195e-05,
   5.095693632028997e-05,
   2.9587761673610657e-05,
   2.1597934392048046e-05]],
 [[101,
   42,
   23,
   50,
   30,
   15,
   90,
   62,
   89,
   8,
   49,
   73,
   28,
   106,
   26,
   84,
   39,
   68,
   66,
   40]])

In [106]:
map_class_to_label(probs, id2label_str, classes)

[{'TEA': 0.7232240438461304,
  'GROCERY': 0.25969767570495605,
  'COFFEE': 0.006111669819802046,
  'HERB': 0.0060967495664954185,
  'EDIBLE_OIL_VEGETABLE': 0.0011588569032028317,
  'BREAD': 0.000731807027477771,
  'SKIN_MOISTURIZER': 0.0005764953093603253,
  'LEGUME': 0.0004914401797577739,
  'SKIN_CLEANING_AGENT': 0.0002766131074167788,
  'BEAUTY': 0.00024830058100633323,
  'HEALTH_PERSONAL_CARE': 0.00020814224262721837,
  'PANTRY': 0.00019130650616716594,
  'DRINKING_CUP': 0.00018323035328648984,
  'VITAMIN': 0.00010350668162573129,
  'DAIRY_BASED_DRINK': 0.00010080193169414997,
  'SAUCE': 9.626983955968171e-05,
  'FOOD_SERVICE_SUPPLY': 5.7873698096955195e-05,
  'NUTRITIONAL_SUPPLEMENT': 5.095693632028997e-05,
  'MEDICATION': 2.9587761673610657e-05,
  'FURNITURE': 2.1597934392048046e-05}]

In [12]:
predictions = list ( zip (labels , torch.softmax(model_res.logits, dim=1).tolist()[0] ) )
predictions = sorted (predictions , key=lambda x:x[1] , reverse =True)

In [13]:
predictions

[('TEA', 0.7232240438461304),
 ('GROCERY', 0.25969767570495605),
 ('COFFEE', 0.006111669819802046),
 ('HERB', 0.0060967495664954185),
 ('EDIBLE_OIL_VEGETABLE', 0.0011588569032028317),
 ('BREAD', 0.000731807027477771),
 ('SKIN_MOISTURIZER', 0.0005764953093603253),
 ('LEGUME', 0.0004914401797577739),
 ('SKIN_CLEANING_AGENT', 0.0002766131074167788),
 ('BEAUTY', 0.00024830058100633323),
 ('HEALTH_PERSONAL_CARE', 0.00020814224262721837),
 ('PANTRY', 0.00019130650616716594),
 ('DRINKING_CUP', 0.00018323035328648984),
 ('VITAMIN', 0.00010350668162573129),
 ('DAIRY_BASED_DRINK', 0.00010080193169414997),
 ('SAUCE', 9.626983955968171e-05),
 ('FOOD_SERVICE_SUPPLY', 5.7873698096955195e-05),
 ('NUTRITIONAL_SUPPLEMENT', 5.095693632028997e-05),
 ('MEDICATION', 2.9587761673610657e-05),
 ('FURNITURE', 2.1597934392048046e-05),
 ('HOME_BED_AND_BATH', 2.1322952306945808e-05),
 ('ACCESSORY_OR_PART_OR_SUPPLY', 1.9804283510893583e-05),
 ('KITCHEN', 1.8700689906836487e-05),
 ('WASTE_BAG', 1.7362539438181557e-

# Quantizing model

# TorchScript

load model as torchscript

In [59]:
script_model = transformers.AutoModelForSequenceClassification.from_pretrained(model_dir, torchscript=True)
script_tokenizer = transformers.AutoTokenizer.from_pretrained(model_dir , torchscript=True)


In [61]:
#query = "test " * 100
res = script_tokenizer.encode_plus(query, return_tensors="pt", padding=True,truncation=True)
res

{'input_ids': tensor([[  101, 27849,  5572,   102]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [62]:
#text = "test " * 100
text = query

# tokenized_text = script_tokenizer.tokenize(text)

# indexed_tokens = script_tokenizer.convert_tokens_to_ids(tokenized_text)
# segments_ids = [1] * len(indexed_tokens)
#tokens_tensor = torch.tensor([indexed_tokens])
#segments_tensors = torch.tensor([segments_ids])

res = script_tokenizer.encode_plus(text, return_tensors="pt", padding=True,truncation=True)


In [63]:
?script_tokenizer.encode_plus

In [56]:
res

{'input_ids': tensor([[  101, 27849,  5572,   102]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [64]:
# Creating a dummy input
tokens_tensor = res['input_ids']
segments_tensors = res['attention_mask']

dummy_input = [tokens_tensor, segments_tensors]

In [65]:
dummy_input

[tensor([[  101, 27849,  5572,   102]]), tensor([[1, 1, 1, 1]])]

In [53]:
tokens_tensor.shape

torch.Size([1, 4])

In [66]:
model_dir_trace ='../artifacts/model/amazon_trace/'

In [67]:
# Creating the trace
traced_model = torch.jit.trace(script_model, [tokens_tensor, segments_tensors])


In [68]:
%%time
res = traced_model(tokens_tensor, segments_tensors)
res

CPU times: user 457 ms, sys: 9.27 ms, total: 466 ms
Wall time: 58.1 ms


(tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
           -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
           -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
           -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
           -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
           -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
            4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
           -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
          -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
           -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
           -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
           -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
           -3.6974,  -6.6184,  -8.4081, 

In [69]:
predictions = list ( zip (labels , torch.softmax(res[0], dim=1).tolist()[0] ) )
predictions = sorted (predictions , key=lambda x:x[1] , reverse =True)
predictions

[('TEA', 0.7232240438461304),
 ('GROCERY', 0.25969767570495605),
 ('COFFEE', 0.006111669819802046),
 ('HERB', 0.0060967495664954185),
 ('EDIBLE_OIL_VEGETABLE', 0.0011588569032028317),
 ('BREAD', 0.000731807027477771),
 ('SKIN_MOISTURIZER', 0.0005764953093603253),
 ('LEGUME', 0.0004914401797577739),
 ('SKIN_CLEANING_AGENT', 0.0002766131074167788),
 ('BEAUTY', 0.00024830058100633323),
 ('HEALTH_PERSONAL_CARE', 0.00020814224262721837),
 ('PANTRY', 0.00019130650616716594),
 ('DRINKING_CUP', 0.00018323035328648984),
 ('VITAMIN', 0.00010350668162573129),
 ('DAIRY_BASED_DRINK', 0.00010080193169414997),
 ('SAUCE', 9.626983955968171e-05),
 ('FOOD_SERVICE_SUPPLY', 5.7873698096955195e-05),
 ('NUTRITIONAL_SUPPLEMENT', 5.095693632028997e-05),
 ('MEDICATION', 2.9587761673610657e-05),
 ('FURNITURE', 2.1597934392048046e-05),
 ('HOME_BED_AND_BATH', 2.1322952306945808e-05),
 ('ACCESSORY_OR_PART_OR_SUPPLY', 1.9804283510893583e-05),
 ('KITCHEN', 1.8700689906836487e-05),
 ('WASTE_BAG', 1.7362539438181557e-

In [70]:
torch.jit.save(traced_model, f"{model_dir_trace}/traced_model.pt")

In [71]:
loaded_model = torch.jit.load( f"{model_dir_trace}/traced_model.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=DistilBertForSequenceClassification
  (distilbert): RecursiveScriptModule(
    original_name=DistilBertModel
    (embeddings): RecursiveScriptModule(
      original_name=Embeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (transformer): RecursiveScriptModule(
      original_name=Transformer
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=TransformerBlock
          (attention): RecursiveScriptModule(
            original_name=MultiHeadSelfAttention
            (dropout): RecursiveScriptModule(original_name=Dropout)
            (q_lin): RecursiveScriptModule(original_name=Linear)
            (k_lin): RecursiveScriptModule(origi

In [72]:
res = loaded_model(tokens_tensor, segments_tensors)
res

(tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
           -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
           -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
           -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
           -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
           -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
            4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
           -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
          -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
           -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
           -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
           -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
           -3.6974,  -6.6184,  -8.4081, 

In [115]:
res

(tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
           -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
           -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
           -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
           -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
           -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
            4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
           -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
          -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
           -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
           -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
           -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
           -3.6974,  -6.6184,  -8.4081, 

[tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
           -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
           -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
           -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
           -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
           -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
            4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
           -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
          -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
           -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
           -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
           -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
           -3.6974,  -6.6184,  -8.4081, 

In [127]:
list(res)

[tensor([[ -7.6677,  -8.9506,  -7.0845,  -5.2786,  -7.1656,  -6.1171,  -9.9482,
           -9.1212,  -2.7499,  -7.2937,  -8.6023,  -8.4928,  -5.4319,  -8.9995,
           -5.5602,  -1.6690,  -6.5178,  -8.3231,  -7.7803,  -9.7673,  -6.5894,
           -8.0236, -10.5098,   0.4534,  -8.5425,  -8.0849,  -3.6514,  -8.6150,
           -3.0538,  -7.3485,  -1.2094,  -8.9694, -10.4731,  -6.9120,  -5.5614,
           -6.1381,  -8.4838,  -9.5964,  -9.0998,  -4.2063,  -5.1919,  -8.1918,
            4.2027,  -7.9540,  -6.4338,  -8.7222,  -7.1057,  -9.4445,  -9.8354,
           -2.9263,   0.4510,  -5.4933,  -5.2048,  -6.2273,  -5.9204,  -8.1743,
          -10.3333,  -7.3300,  -8.0885,  -5.3360, -11.0123,  -7.2948,  -2.0672,
           -8.7549,  -8.7169,  -8.9316,  -4.8772,  -6.4113,  -4.3336, -11.0367,
           -7.4192,  -9.4832,  -6.4128,  -3.0107,  -5.8093,  -6.4888,  -7.1651,
           -7.2252,  -9.9273,  -9.3188,  -7.8169,  -7.1867,  -7.4152,  -9.8344,
           -3.6974,  -6.6184,  -8.4081, 

In [130]:
torch.stack(res).shape

torch.Size([1, 1, 112])

In [74]:
predictions = list ( zip (labels , torch.softmax(res[0], dim=1).tolist()[0] ) )
predictions = sorted (predictions , key=lambda x:x[1] , reverse =True)
predictions

[('TEA', 0.7232240438461304),
 ('GROCERY', 0.25969767570495605),
 ('COFFEE', 0.006111669819802046),
 ('HERB', 0.0060967495664954185),
 ('EDIBLE_OIL_VEGETABLE', 0.0011588569032028317),
 ('BREAD', 0.000731807027477771),
 ('SKIN_MOISTURIZER', 0.0005764953093603253),
 ('LEGUME', 0.0004914401797577739),
 ('SKIN_CLEANING_AGENT', 0.0002766131074167788),
 ('BEAUTY', 0.00024830058100633323),
 ('HEALTH_PERSONAL_CARE', 0.00020814224262721837),
 ('PANTRY', 0.00019130650616716594),
 ('DRINKING_CUP', 0.00018323035328648984),
 ('VITAMIN', 0.00010350668162573129),
 ('DAIRY_BASED_DRINK', 0.00010080193169414997),
 ('SAUCE', 9.626983955968171e-05),
 ('FOOD_SERVICE_SUPPLY', 5.7873698096955195e-05),
 ('NUTRITIONAL_SUPPLEMENT', 5.095693632028997e-05),
 ('MEDICATION', 2.9587761673610657e-05),
 ('FURNITURE', 2.1597934392048046e-05),
 ('HOME_BED_AND_BATH', 2.1322952306945808e-05),
 ('ACCESSORY_OR_PART_OR_SUPPLY', 1.9804283510893583e-05),
 ('KITCHEN', 1.8700689906836487e-05),
 ('WASTE_BAG', 1.7362539438181557e-

In [107]:
with open(f'{model_dir}index_to_name.json','w') as f:
    json.dump(id2label_str,f)

In [108]:
with open(f'{model_dir_trace}index_to_name.json','w') as f:
    json.dump(id2label_str,f)

112

In [51]:
tokenizer.model_max_length

512

In [52]:
tokenizer.max_len_single_sentence

510

In [83]:
tokenizer.save_pretrained(model_dir_trace)

('../artifacts/model/amazon_trace/tokenizer_config.json',
 '../artifacts/model/amazon_trace/special_tokens_map.json',
 '../artifacts/model/amazon_trace/vocab.txt',
 '../artifacts/model/amazon_trace/added_tokens.json',
 '../artifacts/model/amazon_trace/tokenizer.json')

In [109]:
setup_config = {
 "model_name":"pt-original",
 "do_lower_case": tokenizer.do_lower_case,
 "num_labels":len(id2label),
 "save_mode":"original",
 "max_length":tokenizer.model_max_length,
 "captum_explanation": True,
 "embedding_name": "bert",
 "FasterTransformer":False,
 "top_k": 5   

}

setup_config



{'model_name': 'pt-original',
 'do_lower_case': True,
 'num_labels': 112,
 'save_mode': 'original',
 'max_length': 512,
 'captum_explanation': True,
 'embedding_name': 'bert',
 'FasterTransformer': False,
 'top_k': 5}

In [110]:
with open(f'{model_dir}setup_config.json','w') as f:
    json.dump(setup_config,f)

In [111]:
setup_config_trace = {**setup_config}
setup_config_trace['model_name'] = "pt-jit"
setup_config_trace['captum_explanation'] = False
setup_config_trace['save_mode'] = 'jit'
setup_config_trace

{'model_name': 'pt-jit',
 'do_lower_case': True,
 'num_labels': 112,
 'save_mode': 'jit',
 'max_length': 512,
 'captum_explanation': False,
 'embedding_name': 'bert',
 'FasterTransformer': False,
 'top_k': 5}

In [112]:
with open(f'{model_dir_trace}setup_config.json','w') as f:
    json.dump(setup_config_trace,f)